This notebook is the next step in data cleaning and light feature engineering. Unlike the python script that conducted dated cleaning for "obvious" things like fixing data types, (E.g. replacing 19.2% with 0.192) and added measures like calculating lost principle on defaulted loans and monthly debt payments per customer. We also removed a small number of rows due to having missing data, or because they were for loans that still current, so we wouldn't know if the loan would be paid off vs. eventually going into default. 

**High Level Findings (based on the dataset post cleaning)**
* Default rate: 19.18% 
* Average loan Amount: 15,307.37
* Average interest rate: 12.55% 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sn
%matplotlib inline

In [2]:
# set parameters for how many viewable columns there are 

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999



In [3]:
lending = pd.read_csv('data/processed_2015_loan_data_april22.csv')
lending.shape

(396607, 60)

In [4]:
lending.head()

,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,pub_rec_bankruptcies,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,collections_12_mths_ex_med,application_type,acc_now_delinq,chargeoff_within_12_mths,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_il_tl,num_tl_120dpd_2m,pct_tl_nvr_dlq,tot_coll_amt,tot_cur_bal,total_bal_ex_mort,emp_length_months,dti_dec,length_of_credit_history,monthly_income,monthly_debt_payments,updated_monthly_debt_payments,lost_principle,total_payments,post_loan_dti,net_gain
0,20000.0,36 months,0.1485,691.84,C,6,RENT,110000.0,Not Verified,2015-12-01,Fully Paid,credit_card,IL,12.45,0.0,2007-06-01,690.0,0.0,NaN,8.0,0.0,0.0,21374.0,0.845,12.0,24889.01336,24889.01,20000.00,4889.01,0.0,0.0,Individual,0.0,0.0,3.0,5356.0,0.0,102.0,16.0,10.0,0.0,9.0,NaN,0.0,1.0,0.0,100.0,0.0,37491.0,37491.0,72,0.1245,102,9166.666667,1141.250000,1833.090000,0.00,35.975100,0.199973,4889.01
1,20000.0,36 months,0.1577,700.88,D,5,RENT,70000.0,Not Verified,2015-12-01,Fully Paid,house,FL,22.21,0.0,2004-07-01,680.0,0.0,38.0,24.0,0.0,0.0,19077.0,0.366,63.0,21780.58678,21780.59,20000.00,1780.59,0.0,0.0,Individual,0.0,0.0,9.0,2759.0,0.0,137.0,2.0,2.0,1.0,9.0,NaN,5.0,39.0,0.0,92.1,264.0,63456.0,63456.0,60,0.2221,137,5833.333333,1295.583333,1996.463333,0.00,31.076057,0.342251,1780.59
2,10000.0,60 months,0.1797,253.78,D,2,MORTGAGE,55000.0,Not Verified,2015-12-01,Charged Off,credit_card,CO,35.70,0.0,2001-04-01,685.0,0.0,NaN,14.0,0.0,0.0,38623.0,0.780,28.0,5558.20000,5558.20,2687.15,2871.05,0.0,0.0,Individual,0.0,0.0,4.0,20578.0,0.0,176.0,7.0,6.0,6.0,6.0,NaN,0.0,6.0,0.0,100.0,0.0,288087.0,71518.0,24,0.3570,176,4583.333333,1636.250000,1890.030000,7312.85,21.901647,0.412370,-4441.80
3,20000.0,36 months,0.0849,631.26,B,10,MORTGAGE,85000.0,Not Verified,2015-12-01,Fully Paid,major_purchase,SC,17.61,1.0,1999-02-01,705.0,0.0,3.0,8.0,0.0,0.0,826.0,0.057,15.0,21538.50898,21538.51,20000.00,1538.51,0.0,0.0,Individual,0.0,0.0,4.0,17700.0,0.0,55.0,32.0,13.0,3.0,8.0,NaN,1.0,9.0,0.0,93.3,0.0,141601.0,27937.0,120,0.1761,201,7083.333333,1247.375000,1878.635000,0.00,34.119870,0.265219,1538.51
4,10000.0,36 months,0.0649,306.45,A,6,RENT,85000.0,Not Verified,2015-12-01,Fully Paid,credit_card,PA,13.07,0.0,2002-04-01,685.0,1.0,NaN,14.0,1.0,1.0,10464.0,0.345,23.0,10998.97157,10998.97,10000.00,998.97,0.0,0.0,Individual,0.0,0.0,7.0,1997.0,0.0,129.0,1.0,1.0,1.0,1.0,NaN,0.0,3.0,0.0,95.7,8341.0,27957.0,27957.0,72,0.1307,164,7083.333333,925.791667,1232.241667,0.00,35.891570,0.173964,998.97


Get high level summary stats before we proceed further with cleaning and EDA

In [5]:
lending.describe()

,funded_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,pub_rec_bankruptcies,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_il_tl,num_tl_120dpd_2m,pct_tl_nvr_dlq,tot_coll_amt,tot_cur_bal,total_bal_ex_mort,emp_length_months,dti_dec,length_of_credit_history,monthly_income,monthly_debt_payments,updated_monthly_debt_payments,lost_principle,total_payments,post_loan_dti,net_gain
count,396607.000000,396607.000000,396607.000000,396607.000000,3.966070e+05,396607.000000,396607.000000,396607.000000,396607.000000,205025.000000,396607.000000,396607.000000,396607.000000,3.966070e+05,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,3.966070e+05,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,143116.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,3.966070e+05,3.966070e+05,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000,396607.000000
mean,15432.514113,0.125966,446.517832,6.110959,7.850727e+04,19.065596,0.351681,693.606063,0.572229,33.838386,12.030105,0.227359,0.128530,1.786801e+04,0.540277,25.575736,16792.291190,16785.669719,13384.143746,3114.960832,2.121773e+00,0.020322,0.006127,0.010348,4.680979,13299.150461,13.717350,182.348012,13.466399,7.971692,1.676836,6.035272,35.618561,0.524640,8.800019,0.000736,93.923565,252.900748,1.427852e+05,5.343058e+04,73.331504,0.190656,196.642427,6542.272520,1150.506642,1597.024474,2048.370368,37.340225,0.269028,1068.712238
std,8586.000742,0.043243,245.390252,3.577283,7.525791e+04,8.652028,0.933297,29.957265,0.867657,21.959684,5.650937,0.644387,0.380431,2.438064e+04,0.238760,12.108512,10869.118986,10864.638855,8859.976340,3161.258740,1.488623e+01,0.157995,0.086785,0.116421,3.200389,15779.469157,784.327467,90.257651,17.077018,9.018917,1.986882,5.958791,22.591692,1.332438,7.524807,0.029698,8.878326,2232.729788,1.565731e+05,4.959436e+04,42.927394,0.086520,86.219747,6271.492559,747.086392,876.180515,5244.105415,12.436093,0.103904,5829.755112
min,1000.000000,0.053200,14.010000,1.000000,3.800000e+03,0.000000,0.000000,660.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,-5.100000e-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,12.000000,0.000000,36.000000,316.666667,0.000000,31.420000,0.000000,0.000000,0.000623,-35000.000000
25%,9000.000000,0.091700,267.790000,2.000000,4.800000e+04,12.550000,0.000000,670.000000,0.000000,15.000000,8.000000,0.000000,0.000000,6.573000e+03,0.363000,17.000000,8309.014966,8307.370000,6023.450000,1006.730000,0.000000e+00,0.000000,0.000000,0.000000,2.000000,3294.000000,0.000000,119.000000,4.000000,3.000000,0.000000,1.000000,16.000000,0.000000,4.000000,0.000000,90.900000,0.000000,3.225300e+04,2.347900e+04,24.000000,0.125500,137.000000,4000.000000,649.875000,988.763333,0.000000,33.511177,0.191339,490.955000
50%,14025.000000,0.122900,390.830000,6.000000,6.590000e+04,18.510000,0.000000,685.000000,0.000000,30.000000,11.000000,0.000000,0.000000,1.214300e+04,0.544000,24.000000,14359.543590,14352.470000,12000.000000,2005.130000,0.000000e+00,0.000000,0.000000,0.000000,4.000000,7454.000000,0.000000,167.000000,8.000000,5.000000,1.000000,4.000000,32.000000,0.000000,7.000000,0.000000,97.400000,0.000000,8.266800e+04,4.064900e+04,72.000000,0.185100,180.000000,5491.6

Analyze the loan purpose column, as there may be opportunities to condense the categories. 

In [6]:
# count the values in each column 

purpose_count = pd.DataFrame(lending['purpose'].value_counts())

# resetting the index because the values in the purpose column will get set as the index

purpose_df = purpose_count.reset_index()

# fix the true index vs. categories columns 

purpose_df.rename(columns={'purpose':'count'}, inplace = True)
purpose_df.rename(columns={'index':'loan_purpose'}, inplace = True)

# next add % of total 

loan_count = len(lending)

purpose_df['per_of_total'] = purpose_df['count'] / loan_count

purpose_df

,loan_purpose,count,per_of_total
0,debt_consolidation,236525,0.596371
1,credit_card,95842,0.241655
2,home_improvement,23423,0.059058
3,other,17760,0.044780
4,major_purchase,6995,0.017637
5,medical,3598,0.009072
6,car,3275,0.008258
7,small_business,3252,0.008200
8,moving,2268,0.005719
9,vacation,2072,0.005224


* Caveat: all of these purposes are self_reported, so they may not have much value. People could be reporting the category that they think makes it most likely that they'll get the loan, rather than their true purpose. 
* ~83.8% of the customers were in the debt consolidation or credit card category, which are just two different ways of saying the same thing, so we will consolidate those cateogies.  
* ~90% of customers were taking out loans to consolidate debt or for home improvements. 
* The other categories are too small to be relevant from a modeling perspective, will combine the categories as follows, condensing 14 categories into three:
    * debt consolidation & credit card into 'debt_consolidation'
    * consumer_credit made up of home_improvement, major_purchase, car, house, vacation, moving 
    * rest will go into "other" 


In [7]:
# the code to condense categories will hot encode them at the same time 

lending.loc[:, 'debt_consolidation'] = lending.loc[:,'purpose'].apply(lambda x:\
            1 if x == 'debt_consolidation' else (1 if x == 'credit_card' else 0))

lending.loc[:, 'consumer_credit'] = lending.loc[:,'purpose'].apply(lambda x: 1 if\
            x == 'home_improvement' else (1 if x == 'major_purchase' else 1 if x\
                                          == 'car' else 1 if x == 'wedding' else 1 if x == 'moving' else 1 if x\
                                          == 'house' else 1 if x == 'vacation' else  0))

lending.loc[:, 'other'] = lending.loc[:,'purpose'].apply(lambda x: 1 if x\
            == 'other' else (1 if x == 'medical' else 1 if x == 'small_business' else\
                             1 if x == 'renewable_energy' else 1 if x == 'educational' else 0))

In [8]:
lending.head(5)

,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,pub_rec_bankruptcies,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,collections_12_mths_ex_med,application_type,acc_now_delinq,chargeoff_within_12_mths,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_il_tl,num_tl_120dpd_2m,pct_tl_nvr_dlq,tot_coll_amt,tot_cur_bal,total_bal_ex_mort,emp_length_months,dti_dec,length_of_credit_history,monthly_income,monthly_debt_payments,updated_monthly_debt_payments,lost_principle,total_payments,post_loan_dti,net_gain,debt_consolidation,consumer_credit,other
0,20000.0,36 months,0.1485,691.84,C,6,RENT,110000.0,Not Verified,2015-12-01,Fully Paid,credit_card,IL,12.45,0.0,2007-06-01,690.0,0.0,NaN,8.0,0.0,0.0,21374.0,0.845,12.0,24889.01336,24889.01,20000.00,4889.01,0.0,0.0,Individual,0.0,0.0,3.0,5356.0,0.0,102.0,16.0,10.0,0.0,9.0,NaN,0.0,1.0,0.0,100.0,0.0,37491.0,37491.0,72,0.1245,102,9166.666667,1141.250000,1833.090000,0.00,35.975100,0.199973,4889.01,1,0,0
1,20000.0,36 months,0.1577,700.88,D,5,RENT,70000.0,Not Verified,2015-12-01,Fully Paid,house,FL,22.21,0.0,2004-07-01,680.0,0.0,38.0,24.0,0.0,0.0,19077.0,0.366,63.0,21780.58678,21780.59,20000.00,1780.59,0.0,0.0,Individual,0.0,0.0,9.0,2759.0,0.0,137.0,2.0,2.0,1.0,9.0,NaN,5.0,39.0,0.0,92.1,264.0,63456.0,63456.0,60,0.2221,137,5833.333333,1295.583333,1996.463333,0.00,31.076057,0.342251,1780.59,0,1,0
2,10000.0,60 months,0.1797,253.78,D,2,MORTGAGE,55000.0,Not Verified,2015-12-01,Charged Off,credit_card,CO,35.70,0.0,2001-04-01,685.0,0.0,NaN,14.0,0.0,0.0,38623.0,0.780,28.0,5558.20000,5558.20,2687.15,2871.05,0.0,0.0,Individual,0.0,0.0,4.0,20578.0,0.0,176.0,7.0,6.0,6.0,6.0,NaN,0.0,6.0,0.0,100.0,0.0,288087.0,71518.0,24,0.3570,176,4583.333333,1636.250000,1890.030000,7312.85,21.901647,0.412370,-4441.80,1,0,0
3,20000.0,36 months,0.0849,631.26,B,10,MORTGAGE,85000.0,Not Verified,2015-12-01,Fully Paid,major_purchase,SC,17.61,1.0,1999-02-01,705.0,0.0,3.0,8.0,0.0,0.0,826.0,0.057,15.0,21538.50898,21538.51,20000.00,1538.51,0.0,0.0,Individual,0.0,0.0,4.0,17700.0,0.0,55.0,32.0,13.0,3.0,8.0,NaN,1.0,9.0,0.0,93.3,0.0,141601.0,27937.0,120,0.1761,201,7083.333333,1247.375000,1878.635000,0.00,34.119870,0.265219,1538.51,0,1,0
4,10000.0,36 months,0.0649,306.45,A,6,RENT,85000.0,Not Verified,2015-12-01,Fully Paid,credit_card,PA,13.07,0.0,2002-04-01,685.0,1.0,NaN,14.0,1.0,1.0,10464.0,0.345,23.0,10998.97157,10998.97,10000.00,998.97,0.0,0.0,Individual,0.0,0.0,7.0,1997.0,0.0,129.0,1.0,1.0,1.0,1.0,NaN,0.0,3.0,0.0,95.7,8341.0,27957.0,27957.0,72,0.1307,164,7083.333333,925.791667,1232.241667,0.00,35.891570,0.173964,998.97,1,0,0


In [9]:
# fix loan status column 

status_count = pd.DataFrame(lending['loan_status'].value_counts())

# reset and fix index and column labels 
status_df = status_count.reset_index()

status_df.rename(columns={'loan_status':'count'}, inplace = True)
status_df.rename(columns={'index':'loan_status'}, inplace = True)

# next add % of total 

total_loans = len(lending)

status_df['per_of_total'] = status_df['count'] / total_loans 

status_df




,loan_status,count,per_of_total
0,Fully Paid,312420,0.787732
1,Charged Off,74119,0.186883
2,Current,9572,0.024135
3,In Grace Period,253,0.000638
4,Late (31-120 days),156,0.000393
5,Late (16-30 days),76,0.000192
6,Default,11,0.000028


* ~97.5% fall into either fully paid or charged off. 
* Given that we're just trying to identify customers that have either paid off their loans or the loans have been deemed uncollectible (i.e. "charged off") the other statuses aren't relevant
* There is potential value in applying the model on the loans that are either current, in the grace period or have missed payments and then observing what happens to assess how accurate the model is at predicting current customers who will default/would need significant interventions to prevent defaults, but that's beyond the scope of this exercise. 


In [10]:
# isolate just statuses of fully paid or charged off. 

statuses = ['Fully Paid', 'Charged Off']
lending.loan_status.isin(statuses)
lending = lending[lending.loan_status.isin(statuses)]

lending.shape



(386539, 63)

In [11]:
# let's assess the charge off rate with just loans that been deeemed uncollectible
# i.e. "charged off" vs fully paid 

status_count = pd.DataFrame(lending['loan_status'].value_counts())

# reset and fix index and column labels 
status_df = status_count.reset_index()

status_df.rename(columns={'loan_status':'count'}, inplace = True)
status_df.rename(columns={'index':'loan_status'}, inplace = True)

# next add % of total 

total_loans = len(lending)

status_df['per_of_total'] = status_df['count'] / total_loans 

status_df






,loan_status,count,per_of_total
0,Fully Paid,312420,0.80825
1,Charged Off,74119,0.19175


A rather high 19.18% of the loans were defaulted on, which makes this a risky proposition for investors.

If you were to invest 10k in a bucket of loans that roughly approximated the performance of all the loans in this dataset at the average interest rate, a lot of your gains would be offset by the losses on the bad loans. 

A more detailed analysis on potential gains and losses will follow once the data is fully cleaned and more high level EDA completed. 



In [12]:
# need to assess how to handle the NaNs in months since last delinquency
# there are nearly 200k records with NaNs, suggesting this is not a data collection issue
# but is more likely customers that simply don't have a delinquency on their credit report 


test = lending.dropna(subset=['mths_since_last_delinq'])

test.shape




(199926, 63)

In [13]:
# let's look at the default rates in the group without NaNs 


status_count_test = pd.DataFrame(test['loan_status'].value_counts())

# reset and fix index and column labels 
status_df_test = status_count_test.reset_index()

status_df_test.rename(columns={'loan_status':'count'}, inplace = True)
status_df_test.rename(columns={'index':'loan_status'}, inplace = True)

# next add % of total 

total_loans_test = len(test)

status_df_test['per_of_total'] = status_df_test['count'] / total_loans_test 

status_df_test




,loan_status,count,per_of_total
0,Fully Paid,160253,0.801562
1,Charged Off,39673,0.198438


Given that nearly 1/2 of our dataset has NaN values for months since last delinquency, we have to study this column further as dropping all of these rows would probably hurt our analysis, as would filling in the values with the average. We're going to study this more to make a higher quality inference around what these values should be. 

In [14]:
# let's check and see if there are any zero values 

zero_delinq = lending[(lending['mths_since_last_delinq'] == 0)]
zero_delinq.shape



(457, 63)

In [15]:
# let's keep the rows with NaN values for months since last Delinquency 
# and see how they perform as far as paying off thier loans 

# let's look at the default rates in the group without NaNs 

count_null = lending[lending['mths_since_last_delinq'].isnull()]


status_count_null = pd.DataFrame(count_null['loan_status'].value_counts())

# reset and fix index and column labels 
status_df_null = status_count_null.reset_index()


status_df_null.rename(columns={'loan_status':'count'}, inplace = True)
status_df_null.rename(columns={'index':'loan_status'}, inplace = True)

# next add % of total 

total_loans_null = len(count_null)

status_df_null['per_of_total'] = status_df_null['count'] / total_loans_null

status_df_null




,loan_status,count,per_of_total
0,Fully Paid,152167,0.815415
1,Charged Off,34446,0.184585


In [16]:
# since the charge off rates are rather close, let's see if there is a difference in average credit score


count_null['fico_range_low'].mean()



703.0411064609647

In [17]:
test['fico_range_low'].mean()

684.7605864169743

In [18]:
lending['fico_range_low'].mean()

693.5860417706881

The FICO scores are:

* 703 for the customers with NaN for months since last delinquency 
* 684 for the customers without a NaN, nearly all of whom have a value for time since last delinquency
* 693 for the customer group overall. 

The NaN group having a higher FICO does support the idea that since there wasn't a delinquency on their credit report, LC's systems simply didn't enter in a value and thus a NaN was generated. 

In [19]:
# let's look at interest rates for both groups as well 

count_null['int_rate'].mean()



0.1231286405555883

In [20]:
test['int_rate'].mean()

0.1278172303752388

In [21]:
# see if the group with NaN have lower instances of having ever had a significantly past due accounts 

count_null['num_accts_ever_120_pd'].sum()


2690.0

In [22]:
test['num_accts_ever_120_pd'].sum()

201053.0

**Findings Summary**

* Nearly 1/2 of the total dataset has "NaN" for months since last delinquency 
* Default rates for both the NaN and non NaN cohorts are very close at 19.8% and 18.46% respectively
* The credit scores for the group with NaN are nearly an entire tier higher, as they're nearly 20 points higher than the group without NaNs. 
* interest rates for the group without NaNs is about 1/2 a point higher.
* The group without NaNs have had 201,053 instances (for 199k loans) of having an account seriously past due (120 days) vs. only 2,690 for the NaN group of roughly ~184k loans. 

Given the evidence, it's reasonable to presume that the group with NaNs have significantly better credit and payment history and that the NaNs represent not having had a delinquency. The solution for this will be as follows:

* Remove the zeros as such a small number (457) that it won't hurt our analysis 
* fill in all the missing values with zeros 
* subtract 84 from all the other values and store that result in a column called "countdown_zero_delinq", as that way a value of 1 month since last delinquency isn't seen by the model as only slightly better than 1 and a lot worse than 75, when in fact 75 is a lot better than one and 0 is best of all  

Note: using 84 as delinquencies only stay on one's credit report for 84 months

In [23]:

# filter out the zero values 

lending = lending[(lending['mths_since_last_delinq'] != 0)]


# fill in the NaNs with zeroes 

lending.loc[:,'mths_since_last_delinq'].fillna(0, inplace=True)


# create the column with the countdown to zero delinquencies 

lending.loc[:, 'countdown_zero_delinq'] =\
lending.loc[:,'mths_since_last_delinq'].apply(lambda x: 0 if x == 0 else x - 84)


In [24]:
# see how many zeroes are in the months since recent revolving delinquency column 

zero_revol_delinq = lending[(lending['mths_since_recent_revol_delinq'] == 0)]
zero_revol_delinq.shape



(2, 64)

In [25]:
# repeat the process above for the column 'mths_since_recent_revol_delinq' as the same idea applies, if
# not more since there are even fewer zeroes (2 vs. 457)

# filter out the zero values 

lending = lending[(lending['mths_since_recent_revol_delinq'] != 0)]


# fill in the NaNs with zeroes 

lending.loc[:,'mths_since_recent_revol_delinq'].fillna(0, inplace=True)


# create the column with the countdown to zero delinquencies 

lending.loc[:, 'countdown_zero_revol_delinq'] =\
lending.loc[:,'mths_since_recent_revol_delinq'].apply(lambda x: 0 if x == 0 else x - 84)




In [26]:
lending.shape

(386080, 65)

In [27]:
# evaluate the rest of the columns for potential NaNs 


print(" \nCount total NaN at each column in a DataFrame : \n\n", 
      lending.isnull().sum()) 

 
Count total NaN at each column in a DataFrame : 

 funded_amnt                       0
term                              0
int_rate                          0
installment                       0
grade                             0
emp_length                        0
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
purpose                           0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
fico_range_low                    0
inq_last_6mths                    0
mths_since_last_delinq            0
open_acc                          0
pub_rec                           0
pub_rec_bankruptcies              0
revol_bal                         0
revol_util                        0
total_acc                         0
total_pymnt                       0
total_pymnt

In the original pass the following columns all had the exact same number of NaNs 367,120:
* all_util
* inq_last_12m
* max_bal_bc
* open_acc_6m
* open_il_12m
* open_act_il 

Given the consistency of the number of NaNs it stands to reason that this data field wasn't being tracked at the time 90% of this data was generated. Since it's not present for 367k out of 384k records there is no value in adding zero, so I updated the pre-processing file not to select these columns.  


In [28]:
# see the shape of the final data frame 

lending.shape

(386080, 65)

In [29]:
# do another round of describe to see what the updated high level numbers are

lending.describe()


,funded_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,pub_rec_bankruptcies,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_il_tl,num_tl_120dpd_2m,pct_tl_nvr_dlq,tot_coll_amt,tot_cur_bal,total_bal_ex_mort,emp_length_months,dti_dec,length_of_credit_history,monthly_income,monthly_debt_payments,updated_monthly_debt_payments,lost_principle,total_payments,post_loan_dti,net_gain,debt_consolidation,consumer_credit,other,countdown_zero_delinq,countdown_zero_revol_delinq
count,386080.000000,386080.000000,386080.000000,386080.000000,3.860800e+05,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,3.860800e+05,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,3.860800e+05,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,3.860800e+05,3.860800e+05,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000,386080.000000
mean,15307.366478,0.125536,445.786265,6.102082,7.835030e+04,19.008054,0.349205,693.605432,0.575303,17.552518,12.014010,0.228973,0.129945,1.771811e+04,0.538706,25.576326,16521.146714,16514.811016,13238.893420,2981.649551,2.031257e+00,0.020327,0.004784,0.010311,4.699456,13262.583475,12.343926,181.790305,13.376520,7.936687,1.674327,6.025640,12.900000,0.527025,8.799938,0.000658,93.924865,254.913088,1.422298e+05,5.317361e+04,73.224990,0.190081,196.101049,6529.192061,1144.250805,1590.037069,2068.473058,36.819191,0.268446,915.207750,0.837212,0.099715,0.063072,-25.845814,-17.388127
std,8581.671296,0.043296,246.847515,3.577298,7.534039e+04,8.640392,0.931179,30.015812,0.870252,23.168981,5.643891,0.645457,0.382094,2.413611e+04,0.238768,12.120637,10750.914315,10746.779102,8859.566281,3020.294310,1.406705e+01,0.157691,0.076783,0.116233,3.206543,15784.976601,734.823896,90.137581,16.942333,8.976146,1.989637,5.950307,21.877108,1.335718,7.530607,0.027961,8.878989,2244.523049,1.564473e+05,4.930936e+04,42.927577,0.086404,86.136177,6278.365865,743.807738,874.139569,5307.394348,12.161359,0.103852,5794.860137,0.369172,0.299620,0.243093,29.555443,26.826744
min,1000.000000,0.053200,14.010000,1.000000,3.800000e+03,0.000000,0.000000,660.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,-5.100000e-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,12.000000,0.000000,36.000000,316.666667,0.000000,31.420000,0.000000,0.000000,0.000623,-35000.000000,0.000000,0.000000,0.000000,-83.000000,-83.000000
25%,8800.000000,0.091700,266.207500,2.000000,4.800000e+04,12.500000,0.000000,670.000000,0.000000,0.000000,8.000000,0.000000,0.000000,6.511000e+03,0.361000,17.000000,8144.207502,8141.602500,6000.000000,989.107500,0.000000e+00,0.000000,0.000000,0.000000,2.000000,3274.000000,0.000000,119.000000,4.000000,3.000000,0.000000,1.000000,0.000000,0.000000,4.000000,0.000000,90.900000,0.000000,3.199475e+04,2.333775e+04,24.000000,0.125000,136.000000,4000.000000,645.900000,983.338333,0.000000,33.393406,0.190758,462.000000,1.000000,0.000000,0.000000,-55.000000,-37.000000
50%,14000.000000,0.122900,388.130000,6.000000,6.500000e+04,18.440000,0.000000,685.000000,0.000000,4.000

In [30]:
# write the final data frame to csv file for use in the machine learning and additional EDA notebooks

lending.to_csv('data/LC_2015_clean(4)_updated_April2022.csv', date_format='%Y-%m-%d', index=False)

